### Import libraries.

In [62]:
import pandas as pd
import numpy as np
import string
from functions import to_snake_case
from functions import round_up_time
import pickle

#### Data Source [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/datasets/Occupancy+Detection+#)

### Read the dataset into a dataframe object.

In [63]:
df1 = pd.read_csv('../datasets/datatraining.txt')
df2 = pd.read_csv('../datasets/datatest.txt')
df3 = pd.read_csv('../datasets/datatest2.txt')

### Merge all datasets into one large dataframe for cleaning purposes.

In [64]:
df = pd.concat([df1, df2], ignore_index=True)
df = pd.concat([df1, df3], ignore_index=True)

### Check for duplicate observations. 
#### There are none.

In [65]:
df.duplicated().value_counts()

False    17895
dtype: int64

### Check the shape of the dataframe.

In [66]:
df.shape, df1.shape, df2.shape, df3.shape

((17895, 7), (8143, 7), (2665, 7), (9752, 7))

### Peek at the data.

In [67]:
df.head()

,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
0,2015-02-04 17:51:00,23.18,27.2720,426.0,721.25,0.004793,1
1,2015-02-04 17:51:59,23.15,27.2675,429.5,714.00,0.004783,1
2,2015-02-04 17:53:00,23.15,27.2450,426.0,713.50,0.004779,1
3,2015-02-04 17:54:00,23.15,27.2000,426.0,708.25,0.004772,1
4,2015-02-04 17:55:00,23.10,27.2000,426.0,704.50,0.004757,1


In [68]:
df1.describe()

,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
count,8143.000000,8143.000000,8143.000000,8143.000000,8143.000000,8143.000000
mean,20.619084,25.731507,119.519375,606.546243,0.003863,0.212330
std,1.016916,5.531211,194.755805,314.320877,0.000852,0.408982
min,19.000000,16.745000,0.000000,412.750000,0.002674,0.000000
25%,19.700000,20.200000,0.000000,439.000000,0.003078,0.000000
50%,20.390000,26.222500,0.000000,453.500000,0.003801,0.000000
75%,21.390000,30.533333,256.375000,638.833333,0.004352,0.000000
max,23.180000,39.117500,1546.333333,2028.500000,0.006476,1.000000


In [69]:
df2.describe()

,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
count,2665.000000,2665.000000,2665.000000,2665.000000,2665.000000,2665.000000
mean,21.433876,25.353937,193.227556,717.906470,0.004027,0.364728
std,1.028024,2.436842,250.210906,292.681718,0.000611,0.481444
min,20.200000,22.100000,0.000000,427.500000,0.003303,0.000000
25%,20.650000,23.260000,0.000000,466.000000,0.003529,0.000000
50%,20.890000,25.000000,0.000000,580.500000,0.003815,0.000000
75%,22.356667,26.856667,442.500000,956.333333,0.004532,1.000000
max,24.408333,31.472500,1697.250000,1402.250000,0.005378,1.000000


In [70]:
df3.describe()

,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
count,9752.000000,9752.000000,9752.000000,9752.000000,9752.000000,9752.000000
mean,21.001768,29.891910,123.067930,753.224832,0.004589,0.210111
std,1.020693,3.952844,208.221275,297.096114,0.000531,0.407408
min,19.500000,21.865000,0.000000,484.666667,0.003275,0.000000
25%,20.290000,26.642083,0.000000,542.312500,0.004196,0.000000
50%,20.790000,30.200000,0.000000,639.000000,0.004593,0.000000
75%,21.533333,32.700000,208.250000,831.125000,0.004998,0.000000
max,24.390000,39.500000,1581.000000,2076.500000,0.005769,1.000000


#### Round up timestamp with '59' for a clear timestamp pattern (these may have caused some issues with ARIMA)

In [71]:
df['date'] = df['date'].map(lambda x: round_up_time(x))

In [72]:
len([timestamp for timestamp in df['date'] if ':59:59' in timestamp])

0

In [73]:
len([timestamp for timestamp in df['date'] if ':59' in timestamp[16:]])

0

In [74]:
df1['date'] = df1['date'].map(lambda x: round_up_time(x))
df2['date'] = df2['date'].map(lambda x: round_up_time(x))
df3['date'] = df3['date'].map(lambda x: round_up_time(x))

### Check for missing values.
#### There are none.

In [75]:
df.isnull().sum()

date             0
Temperature      0
Humidity         0
Light            0
CO2              0
HumidityRatio    0
Occupancy        0
dtype: int64

### Check the data types for each column.
#### Notice date is not in DateTime format.  
#### We will need to convert to allow time series indexing.

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17895 entries, 0 to 17894
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           17895 non-null  object 
 1   Temperature    17895 non-null  float64
 2   Humidity       17895 non-null  float64
 3   Light          17895 non-null  float64
 4   CO2            17895 non-null  float64
 5   HumidityRatio  17895 non-null  float64
 6   Occupancy      17895 non-null  int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 978.8+ KB


### Rename all columns to follow snake-case conventions.

In [77]:
df.columns = df.columns.map(lambda x: to_snake_case(x))

In [78]:
df1.columns = df1.columns.map(lambda x: to_snake_case(x))
df2.columns = df2.columns.map(lambda x: to_snake_case(x))
df3.columns = df3.columns.map(lambda x: to_snake_case(x))

In [79]:
df.head()

,date,temperature,humidity,light,co2,humidity_ratio,occupancy
0,2015-02-04 17:51:00,23.18,27.2720,426.0,721.25,0.004793,1
1,2015-02-04 17:52:00,23.15,27.2675,429.5,714.00,0.004783,1
2,2015-02-04 17:53:00,23.15,27.2450,426.0,713.50,0.004779,1
3,2015-02-04 17:54:00,23.15,27.2000,426.0,708.25,0.004772,1
4,2015-02-04 17:55:00,23.10,27.2000,426.0,704.50,0.004757,1


### Class value counts for each dataset.

In [80]:
df['occupancy'].value_counts(normalize=True)

0    0.78888
1    0.21112
Name: occupancy, dtype: float64

In [81]:
df1['occupancy'].value_counts(normalize=True)

0    0.78767
1    0.21233
Name: occupancy, dtype: float64

In [82]:
df2['occupancy'].value_counts(normalize=True)

0    0.635272
1    0.364728
Name: occupancy, dtype: float64

In [83]:
df3['occupancy'].value_counts(normalize=True)

0    0.789889
1    0.210111
Name: occupancy, dtype: float64

### Sort the observations by the date column.
#### We need the date column sorted in ascending order in preparation for time series indexing.

In [84]:
df.sort_values(by='date', inplace=True)

In [85]:
df1.sort_values(by='date', inplace=True)
df2.sort_values(by='date', inplace=True)
df3.sort_values(by='date', inplace=True)

In [86]:
df.head()

,date,temperature,humidity,light,co2,humidity_ratio,occupancy
0,2015-02-04 17:51:00,23.18,27.2720,426.0,721.25,0.004793,1
1,2015-02-04 17:52:00,23.15,27.2675,429.5,714.00,0.004783,1
2,2015-02-04 17:53:00,23.15,27.2450,426.0,713.50,0.004779,1
3,2015-02-04 17:54:00,23.15,27.2000,426.0,708.25,0.004772,1
4,2015-02-04 17:55:00,23.10,27.2000,426.0,704.50,0.004757,1


In [87]:
df.tail()

,date,temperature,humidity,light,co2,humidity_ratio,occupancy
17890,2015-02-18 09:15:00,20.815,27.7175,429.75,1505.25,0.004213,1
17891,2015-02-18 09:16:00,20.865,27.7450,423.50,1514.50,0.004230,1
17892,2015-02-18 09:17:00,20.890,27.7450,423.50,1521.50,0.004237,1
17893,2015-02-18 09:18:00,20.890,28.0225,418.75,1632.00,0.004279,1
17894,2015-02-18 09:19:00,21.000,28.1000,409.00,1864.00,0.004321,1


### Save the cleaned dataset to a csv file for later use.

In [88]:
df.to_csv('../datasets/occupancy.csv', index=False)

In [89]:
df1.to_csv('../datasets/train.csv', index=False)
df2.to_csv('../datasets/test.csv', index=False)
df3.to_csv('../datasets/test2.csv', index=False)

### Create time series index using the date column.

#### Convert the date column from object/string type to DateTime type.

In [90]:
df['date'] = df['date'].map(lambda x: pd.to_datetime(x))

In [91]:
df1['date'] = df1['date'].map(lambda x: pd.to_datetime(x))
df2['date'] = df2['date'].map(lambda x: pd.to_datetime(x))
df3['date'] = df3['date'].map(lambda x: pd.to_datetime(x))

In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17895 entries, 0 to 17894
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            17895 non-null  datetime64[ns]
 1   temperature     17895 non-null  float64       
 2   humidity        17895 non-null  float64       
 3   light           17895 non-null  float64       
 4   co2             17895 non-null  float64       
 5   humidity_ratio  17895 non-null  float64       
 6   occupancy       17895 non-null  int64         
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 1.1 MB


In [93]:
df.set_index('date', inplace=True)

In [94]:
df1.set_index('date', inplace=True)
df2.set_index('date', inplace=True)
df3.set_index('date', inplace=True)

In [95]:
df.head()

,temperature,humidity,light,co2,humidity_ratio,occupancy
date,,,,,,
2015-02-04 17:51:00,23.18,27.2720,426.0,721.25,0.004793,1
2015-02-04 17:52:00,23.15,27.2675,429.5,714.00,0.004783,1
2015-02-04 17:53:00,23.15,27.2450,426.0,713.50,0.004779,1
2015-02-04 17:54:00,23.15,27.2000,426.0,708.25,0.004772,1
2015-02-04 17:55:00,23.10,27.2000,426.0,704.50,0.004757,1


In [96]:
df.sort_index(inplace=True)

In [97]:
df1.sort_index(inplace=True)
df2.sort_index(inplace=True)
df3.sort_index(inplace=True)

In [98]:
df.head()

,temperature,humidity,light,co2,humidity_ratio,occupancy
date,,,,,,
2015-02-04 17:51:00,23.18,27.2720,426.0,721.25,0.004793,1
2015-02-04 17:52:00,23.15,27.2675,429.5,714.00,0.004783,1
2015-02-04 17:53:00,23.15,27.2450,426.0,713.50,0.004779,1
2015-02-04 17:54:00,23.15,27.2000,426.0,708.25,0.004772,1
2015-02-04 17:55:00,23.10,27.2000,426.0,704.50,0.004757,1


In [99]:
df.tail()

,temperature,humidity,light,co2,humidity_ratio,occupancy
date,,,,,,
2015-02-18 09:15:00,20.815,27.7175,429.75,1505.25,0.004213,1
2015-02-18 09:16:00,20.865,27.7450,423.50,1514.50,0.004230,1
2015-02-18 09:17:00,20.890,27.7450,423.50,1521.50,0.004237,1
2015-02-18 09:18:00,20.890,28.0225,418.75,1632.00,0.004279,1
2015-02-18 09:19:00,21.000,28.1000,409.00,1864.00,0.004321,1


### Add a column to indicate if it is a week day (M-F) or a weekend day (Sat or Sun).

In [100]:
df['weekday'] = df.index.astype(str)
df1['weekday'] = df1.index.astype(str)
df2['weekday'] = df2.index.astype(str)
df3['weekday'] = df3.index.astype(str)
df['weekday'] = df['weekday'].map(lambda x: 0 if x[8:10] in ['07', '08', '14', '15'] else 1)
df1['weekday'] = df1['weekday'].map(lambda x: 0 if x[8:10] in ['07', '08', '14', '15'] else 1)
df2['weekday'] = df2['weekday'].map(lambda x: 0 if x[8:10] in ['07', '08', '14', '15'] else 1)
df3['weekday'] = df3['weekday'].map(lambda x: 0 if x[8:10] in ['07', '08', '14', '15'] else 1)

### Impute light variable outliers with the mean for its corresponding class.

In [101]:
df4 = df.copy()

In [102]:
light_1000 = df4[df4['light'] > 1000]
light_1000

,temperature,humidity,light,co2,humidity_ratio,occupancy,weekday
date,,,,,,,
2015-02-07 09:42:00,20.700,18.8900,1546.333333,455.333333,0.002845,0,0
2015-02-07 09:43:00,20.745,18.8900,1451.750000,453.000000,0.002853,0,0
2015-02-12 09:46:00,22.772,25.2900,1380.000000,1202.500000,0.004333,1,1
2015-02-12 09:47:00,22.790,25.2900,1581.000000,1211.500000,0.004338,1,1
2015-02-12 09:48:00,22.790,25.1000,1010.500000,1215.500000,0.004305,1,1
2015-02-13 09:49:00,22.500,24.9175,1021.250000,577.500000,0.004198,1,1


In [103]:
avg_light_by_class = df4.groupby('occupancy')['light'].mean()
avg_light_by_class

occupancy
0     26.185953
1    477.431873
Name: light, dtype: float64

In [104]:
for idx in light_1000.index:
    df4.loc[idx, 'light'] = avg_light_by_class[df4.loc[idx, 'occupancy']]

In [105]:
df4[df4['light'] > 1000]

,temperature,humidity,light,co2,humidity_ratio,occupancy,weekday
date,,,,,,,


### Pickle the data for later use. This will preserve the time series indexing.

In [106]:
df.to_pickle('../datasets/occupancy.p')

In [107]:
df1.to_pickle('../datasets/train.p')
df2.to_pickle('../datasets/test.p')
df3.to_pickle('../datasets/test2.p')

In [108]:
df4.to_pickle('../datasets/occupancy2.p')